# Programme de Matching: Version 2 Nouvelles règles de gestion

## Algorithme

### origin

Comme la taille de la donnée est trop élevée, il faut prendre un sous échantillon pour faire la siretisation. Le sous échantillonage se fait avec l'origine. 

Input:
- INSEE:
    - NEW: `data/input/INSEE/NEW/insee_1745311_NEW.csv`
    - Initial, Partiel, EVT: `data/input/INSEE/InitialPartielEVT/insee_8272605_InitialPartielEVT.csv`
- INPI:
    - NEW: `data/input/SIREN_INPI/NEW/inpi_initial_partiel_evt_new_ets_status_final_NEW_0.csv`
    - Initial, Partiel, EVT: `data/input/SIREN_INPI/InitialPartielEVT/inpi_initial_partiel_evt_new_ets_status_final_InitialPartielEVT_0`

Output:
- `data/output/` + `ORIGIN`
    - contient les matches
- `data/input/INPI/special_treatment/`+ `ORIGIN`
    - contient les non matches
- `data/logs/`+ `ORIGIN`
    - contient les logs
    
       

In [1]:
import os, re
os.chdir('../')
current_dir = os.getcwd()
from inpi_insee import siretisation
import pandas as pd

%load_ext autoreload
%autoreload 2

param = {
    #'insee': 'data/input/INSEE/InitialPartielEVTNEW/insee_1557220_InitialPartielEVTNEW.csv' ### PP
    'insee': 'data/input/INSEE/InitialPartielEVTNEW/insee_9368683_InitialPartielEVTNEW.csv'  ### ETS
    #'insee': 'data/input/INSEE/NEW/insee_1745311_NEW.csv' ### ETS
}
# 4824158 SIREN a trouver!
al_siret = siretisation.siretisation_inpi(param)

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#from itertools import compress, product

#def combinations(items):
#    return ( set(compress(items,mask)) for 
#            mask in product(*[[0,1]]*len(items)))

#all_list = ['ncc',
#             'Code_Postal','Code_Commune',
#             'INSEE','digit_inpi']
#test = list(combinations(items = all_list))[1:]
#sort_list = sorted(test[1:], key=lambda k: len(k), reverse=True) 

In [4]:
list_inpi = ['ncc','code_postal_matching','code_commune','voie_matching','numero_voie_matching',
             'date_début_activité', 'status_admin', 'status_ets']

list_insee = ['libelleCommuneEtablissement',
            'codePostalEtablissement', 'codeCommuneEtablissement',
            'typeVoieEtablissement','numeroVoieEtablissement',
             'dateCreationEtablissement', 'etatAdministratifEtablissement', 'etablissementSiege']

sort_list = [
 {'ncc', 'code_postal_matching', 'code_commune', 'voie_matching', 'numero_voie_matching',
  'date_début_activité', 'status_admin', 'status_ets'},
 {'ncc', 'code_postal_matching', 'code_commune', 'voie_matching',
  'date_début_activité', 'status_admin', 'status_ets'},
 {'ncc', 'code_postal_matching', 'code_commune', 'numero_voie_matching',
 'date_début_activité', 'status_admin', 'status_ets'},
 {'ncc', 'code_postal_matching', 'code_commune','date_début_activité', 'status_admin', 'status_ets'},   
 {'ncc', 'code_postal_matching','date_début_activité', 'status_admin', 'status_ets'},
 {'ncc', 'date_début_activité', 'status_admin', 'status_ets'},
 {'code_postal_matching', 'date_début_activité', 'status_admin', 'status_ets'},
 {'code_commune', 'date_début_activité', 'status_admin', 'status_ets'},
]
len(sort_list)

8

In [5]:
list_possibilities = []
for i in sort_list:
    left =[]
    right = []
    for j in i:
        left.append(j)
        right.append(list_insee[list_inpi.index(j)])
    left.insert(0,'siren')
    right.insert(0,'siren')
    
    dic_ = {
    'match':{
        'inpi':left,
        'insee':right,
    }
}
    list_possibilities.append(dic_)
list_possibilities

[{'match': {'inpi': ['siren',
    'ncc',
    'code_commune',
    'voie_matching',
    'status_admin',
    'code_postal_matching',
    'date_début_activité',
    'numero_voie_matching',
    'status_ets'],
   'insee': ['siren',
    'libelleCommuneEtablissement',
    'codeCommuneEtablissement',
    'typeVoieEtablissement',
    'etatAdministratifEtablissement',
    'codePostalEtablissement',
    'dateCreationEtablissement',
    'numeroVoieEtablissement',
    'etablissementSiege']}},
 {'match': {'inpi': ['siren',
    'ncc',
    'code_commune',
    'voie_matching',
    'status_admin',
    'code_postal_matching',
    'date_début_activité',
    'status_ets'],
   'insee': ['siren',
    'libelleCommuneEtablissement',
    'codeCommuneEtablissement',
    'typeVoieEtablissement',
    'etatAdministratifEtablissement',
    'codePostalEtablissement',
    'dateCreationEtablissement',
    'etablissementSiege']}},
 {'match': {'inpi': ['siren',
    'ncc',
    'code_commune',
    'status_admin',
    'code_

Indiquer le fichiers a siretiser. Si pas en local, le télécharger depuis le S3

In [ ]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import os
bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/programme_matching/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata') 
athena = service_athena.connect_athena(client = client,
                      bucket = 'calfdata') 

In [ ]:
query = """
SELECT * 
FROM ets_preparation_python_lib1   
"""

In [ ]:
output = athena.run_query(
    query=query,
    database='inpi',
    s3_output='INPI/sql_output'
)

In [ ]:
table = 'ets_preparation_python_lib1_1'
source_key = "{}/{}.csv".format(
                        'INPI/sql_output',
                        output['QueryExecutionId']
                               )

destination_key = "{}/{}.csv".format(
                        'INPI/TC_1/02_preparation_donnee/ETS_TEMP',
                        table
                    )


In [ ]:
results = s3.copy_object_s3(
                        source_key = source_key,
                        destination_key = destination_key,
                        remove = False
                    )

In [ ]:
#s3.download_file(
#    key= destination_key)

In [6]:
import shutil
#try:
#    os.remove("data/input/INPI/InitialPartielEVTNEW/ets_preparation_python_1.csv")
#except:
#    pass
#os.getcwd()

In [ ]:
#filename = 'ets_preparation_python_lib1_1'
#shutil.move("{}.csv".format(filename),
#            "data/input/INPI/InitialPartielEVTNEW")

Il faut prendre l'`origin` et `filename` que l'on souhaite sitetiser

In [ ]:
import glob
files = glob.glob('data/logs/InitialPartielEVTNEW/*')
for f in files:
    os.remove(f)

In [7]:
origin = "InitialPartielEVTNEW"
filename = "ets_preparation_python_lib1" ####ETS
#filename = "inpi_initial_partiel_evt_new_ets_status_final_InitialPartielEVTNEW"
#origin = "NEW"
#filename = "inpi_initial_partiel_evt_new_ets_status_final_NEW"
#### make dir
parent_dir = 'data/output/'
parent_dir_1 = 'data/input/INPI/special_treatment/'
parent_dir_2 = 'data/logs/'

for d in [parent_dir,parent_dir_1,parent_dir_2]:
    path = os.path.join(d, origin) 
    try:
        os.mkdir(path) 
    except:
        pass

# Parametres et fonctions

- `import_dask`: Charge csv en Dask DataFrame pour clusteriser les calculs 

In [8]:
def split_duplication(df):
        """
        Split un dataframe si l'index (la variable, pas l'index) contient des
        doublons.

        L'idée est de distinguer les doublons resultants du merge avec l'INSEE

        Args:
        - df: Pandas dataframe contenant au moins une variable "index"

        Returns:
        - Un Dictionary avec:
            - not_duplication: Dataframe ne contenant pas les doublons
            - duplication: Dataframe contenant les doublons
            - report_dup: Une Serie avec le nombres de doublons
        """
        if 'count_duplicates_' in df.columns:
            df = df.drop(columns = 'count_duplicates_')

        df = df.merge(
            (df
                .groupby('index')['index']
                .count()
                .rename('count_duplicates_')
                .reset_index()
                )
                )
        
        dic_ = {
            'not_duplication':df[df['count_duplicates_'].isin([1])],
            'duplication' : df[~df['count_duplicates_'].isin([1])],
            'report_dup':df[
            ~df['count_duplicates_'].isin([1])
            ]['count_duplicates_'].value_counts()
            }

        return dic_

In [9]:
def find_regex(regex, test_str):
        """
        Performe une recherche regex entre deux colonnes.

        Args:
        - regex: string: charactère contenant la formule regex
        - test_str: String: String contenant le regex a trouver

        Return:
        Boolean, True si le regex est trouvé, sinon False

        """
        try:
            matches = re.search(regex, test_str)
            if matches:
                return True
            else:
                return False
        except:
            return False

In [10]:
inpi_col = ['siren',
            'code_greffe',
            'nom_greffe',
            'numero_gestion',
            'id_etablissement',
            'origin',
            'file_timestamp',
            'date_greffe',
            #'nom_commercial',
            'enseigne',
            'libelle_evt',
            'last_libele_evt',
            'index',
            'type',
            'status_admin', 'status_ets',
            'code_postal_matching',
            #'ville',
            'code_commune',
            #'pays',
            #'count_initial_inpi',
            'ncc',
            'adresse_new_clean_reg',
            'adress_new',
            'voie_matching',
            'date_début_activité',
            'numero_voie_matching',
            #'len_digit_address_inpi',
            #'list_digit_inpi'
            ]

#['count_initial_inpi', 'list_digit_inpi', 'pays', 'len_digit_address_inpi', 'nom_commercial', 'ville']

inpi_dtype = {
    'siren': 'object',
    'code_greffe': 'object',
            'nom_greffe': 'object',
            'numero_gestion': 'object',
            'id_etablissement': 'object',
            'origin': 'object',
            'file_timestamp': 'object',
            'date_greffe': 'object',
    #'nom_commercial': 'object',
            'enseigne': 'object',
            'libelle_evt': 'object',
    'id_etablissement': 'object',
    'index': 'object',
    'type': 'object',
    'status_admin': 'object',
    'status_ets': 'object',
    'code_postal_matching': 'object',
    'ville': 'object',
    'code_commune': 'object',
    'pays': 'object',
    #'count_initial_inpi': 'int',
    'ncc': 'object',
    'adresse_new_clean_reg': 'object',
    'adress_new':'object',
    'voie_matching': 'object',
    'date_début_activité': 'object',
    'numero_voie_matching': 'object',
    #'len_digit_address_inpi':'object'
}

insee_col = ['siren',
         'siret',
         'dateCreationEtablissement',
         "etablissementSiege",
         "etatAdministratifEtablissement",
         'complementAdresseEtablissement',
         'numeroVoieEtablissement',
         'indiceRepetitionEtablissement',
         'typeVoieEtablissement',
         'libelleVoieEtablissement',
         'codePostalEtablissement',
         'libelleCommuneEtablissement',
         'libelleCommuneEtrangerEtablissement',
         'distributionSpecialeEtablissement',
         'codeCommuneEtablissement',
         'codeCedexEtablissement',
         'libelleCedexEtablissement',
         'codePaysEtrangerEtablissement',
         'libellePaysEtrangerEtablissement',
         'count_initial_insee',
             'len_digit_address_insee',
             'list_digit_insee',
            "enseigne1Etablissement",
            "enseigne2Etablissement",
            "enseigne3Etablissement"]

insee_dtype = {
             'siren': 'object',
             'siret': 'object',
             "etablissementSiege": "object",
             "etatAdministratifEtablissement": "object",
             'dateCreationEtablissement': 'object',
             'complementAdresseEtablissement': 'object',
             'numeroVoieEtablissement': 'object',
             'indiceRepetitionEtablissement': 'object',
             'typeVoieEtablissement': 'object',
             'libelleVoieEtablissement': 'object',
             'codePostalEtablissement': 'object',
             'libelleCommuneEtablissement': 'object',
             'libelleCommuneEtrangerEtablissement': 'object',
             'distributionSpecialeEtablissement': 'object',
             'codeCommuneEtablissement': 'object',
             'codeCedexEtablissement': 'object',
             'libelleCedexEtablissement': 'object',
             'codePaysEtrangerEtablissement': 'object',
             'libellePaysEtrangerEtablissement': 'object',
             'count_initial_insee': 'int',
             'len_digit_address_insee':'object',
            "enseigne1Etablissement":'object',
            "enseigne2Etablissement":'object',
            "enseigne3Etablissement":'object'
         }

In [11]:
## Calcul nb siren/siret
upper_word = pd.read_csv('data/input/Parameters/upper_stop.csv'
        ).iloc[:,0].to_list()

In [12]:
voie = (pd.read_csv('data/input/Parameters/voie.csv').assign(upper = lambda x: 
             x['possibilite'].str.isupper()).loc[lambda x: 
                                                 x['upper'].isin([True])])

In [13]:
import nltk
import dask.dataframe as dd
import numpy as np
import sidetable
def jackard_distance(inpi, insee):
    """
    
    """
    
    
    
    try:
        w1 = set(inpi)
        w2 = set(insee)
        return nltk.jaccard_distance(w1, w2)
    except:
        pass

In [14]:
def edit_distance(inpi, insee):
    """
    
    """
    
    try:
        return nltk.edit_distance(inpi, insee)
    except:
        pass

In [15]:
def count_conformite_test(df):
    """
    """
    
    dic_ = {
       # 'test_siege_siege':
    #df['test_siege_siege'].value_counts(),
        
    #    'test_siege_pri_sec':
    #df['test_siege_pri_sec'].value_counts(),
        
    #    'divergence_siege':
    #df['divergence_siege'].value_counts(),
        
    #    'test_etb_ferme':
    #df['test_status_ets_ferme'].value_counts(),
        
    #    'test_etb_ouvert':
    #df['test_status_ets_ouvert'].value_counts(),
        
    #    'divergence_etatadmin':
    #df['divergence_etatadmin'].value_counts(),
        
        'test_join_address':
    df['test_join_address'].value_counts(),
        
        'test_address_libelle':
    df['test_address_libelle'].value_counts(),
        
        'test_address_complement':
    df['test_address_complement'].value_counts(),
        
        
    #    'test_debut_activite_egal':
    #(df.loc[lambda x: 
    #  ~x['date_début_activité'].isin([np.datetime64('NaT')])
    # ]['test_date_egal']
 #.value_counts()
#),
        
 # 'test_debut_activite_sup':      
 #   (df.loc[lambda x: 
 #     ~x['date_début_activité'].isin([np.datetime64('NaT')])
 #    ]['test_date_sup']
 #.value_counts()
#),
        'jacquard':      
    (df['jacquard'].describe()
),
        'edit':      
    (df['edit'].describe())
    }
    return dic_

# Règle de gestion

Dans la première étape, nous allons merger les données de l'INSEE avec les données de l'INPI. Dans le cadre de ce notebook, nous allons utiliser uniquement les variables de matching suivantes:

- Siren
- Numéro de voie
- Code postal
- type de voie
- Code commune
- Ville

``` 
'siren', 'digit_inpi', 'code_postal_matching', 'INSEE', 'code_commune', 'ncc' 
``` 

Des lors que la table a été matché et soustrait des index qui n'ont pas de correspondance a l'INSEE, nous pouvons effectuer les tests suivants:

## Test Adresse

- `test_address_libelle` :
    - Si un des mots contenus dans la variable `adresse_new_clean_reg` est présente dans la variable `libelleVoieEtablissement` alors True
- `test_address_complement`:
    - Si un des mots contenus dans la variable `adresse_new_clean_reg` est présente dans la variable `libelleVoieEtablissement` alors True
- `test_join_address`:
    - Si `test_address_libelle` et `test_address_complement` sont égales a True, alors True
- `jacquard`:
    - Calcul de la distance entre `adresse_inpi_clean` et `adress_insee_reconstitue`

## Test Siege

- `test_siege_principal`:
    - Si `type` contient `SIE` ou `SEP`  et `etablissementSiege` est égal a True, alors True
- `test_siege_secondaire`:
    - Si `type` contient `SEC` ou `PRI` et `etablissementSiege` est égal a False, alors True
- `divergence_siege`:
    - Si `test_siege_principal` est égale à False et `test_siege_secondaire` est égal à False, alors True

## Test Etat 

- `test_status_ets_ferme`:
    - Si `last_libele_evt` est égale à `Etablissement supprimé` et `etatAdministratifEtablissement` est égal à `F` alors True
- `test_status_ets_ouvert`:
    - Si `last_libele_evt` est égale à `Etablissement ouvert` et `etatAdministratifEtablissement` est égal à `A` alors True
- `divergence_etatadmin`: Si `test_status_ets_ferme` est égal à False et `test_status_ets_ouvert` est égal à False, alors True

## Test Date

- `test_date_egal`:
    -  `date_début_activité` est égale à `dateCreationEtablissement` alors True 
- `test_date_sup`:
    - `date_début_activité` est supérieure à `dateCreationEtablissement` alors True 


In [ ]:
list_possibilities[0]['match']['inpi']

In [ ]:
list_possibilities[0]['match']['insee']

Dans un premier temps, on import et on merge les dataframes Insee et INPI. Pour rappel, il y a 10,648,546 d'observations à siretiser. Pour le vérifier, il suffit de regarder la variable index, qui est le numéro de la ligne dans la base initiale de l'INPI.

A noté que nous convertissons la variable `date_début_activité` en format date, car nous avons besoin de comparer cette variable avec `dateCreationEtablissement` lors de nos tests

In [16]:
df_ets = 'data/input/INPI/{0}/{1}_{2}.csv'.format(origin, filename, 1)

inpi = (al_siret.import_dask(file=df_ets,
                                usecols=inpi_col,
                                dtype=inpi_dtype,
                                parse_dates=False)
       )
insee = al_siret.import_dask(
        file=al_siret.insee,
        usecols=insee_col,
        dtype=insee_dtype,
        #parse_dates = ['dateCreationEtablissement']
)

temp = inpi.merge(insee,
                          how='left',
                          left_on=list_possibilities[0]['match']['inpi'],
                          right_on= list_possibilities[0]['match']['insee'],
                          indicator=True,
                          suffixes=['_insee', '_inpi'])
temp = temp.compute()

[########################################] | 100% Completed |  2min 29.4s


Il est assez simple de voir que le merge a abouti a la création de doublon 

In [17]:
temp.shape[0] - temp['index'].max() + 1

9096

Nous allons appliquer des règles de gestion sur les combinaisons matchées (les `both`)

In [18]:
temp['_merge'].value_counts()

left_only     9291435
both          1366205
right_only          0
Name: _merge, dtype: int64

In [19]:
to_check = (temp[temp['_merge']
                 .isin(['both'])]
            .drop(columns= '_merge')
            .merge(voie, left_on = 'typeVoieEtablissement', right_on ='INSEE', how = 'left')
            #### création addresse
            .assign(
                numeroVoieEtablissement_ = lambda x: x['numeroVoieEtablissement'].fillna(''),
                possibilite = lambda x: x['possibilite'].fillna('')
                   )
            .assign(
            date_début_activité = lambda x: pd.to_datetime(
            x['date_début_activité'], errors = 'coerce'),   
        adresse_insee_clean=lambda x: x['libelleVoieEtablissement'].str.normalize(
                'NFKD')
            .str.encode('ascii', errors='ignore')
            .str.decode('utf-8')
            .str.replace('[^\w\s]|\d+', ' ')
            .str.upper(),
    adress_insee_reconstitue = lambda x: 
            x['numeroVoieEtablissement_'] + ' '+ \
            x['possibilite'] + ' ' + \
            x['libelleVoieEtablissement']   
        )
            .assign(
            
        adresse_insee_clean = lambda x: x['adresse_insee_clean'].apply(
        lambda x:' '.join([word for word in str(x).split() if word not in
        (upper_word)])),
            
        adresse_inpi_reconstitue = lambda x: x['adress_new'].apply(
        lambda x:' '.join([word for word in str(x).split() if word not in
        (upper_word)])),   
            
        adress_insee_reconstitue = lambda x: x['adress_insee_reconstitue'].apply(
        lambda x:' '.join([word for word in str(x).split() if word not in
        (upper_word)])), 
                enseigne = lambda x: x['enseigne'].str.upper()
        )
            )

## Test 1: address
df_2 = dd.from_pandas(to_check, npartitions=10)
df_2['test_address_libelle'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    find_regex(
                     x['adresse_new_clean_reg'],
                     x['libelleVoieEtablissement']), axis=1)
                     ).compute()

df_2['test_address_complement'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    find_regex(
                     x['adresse_new_clean_reg'],
                     x['complementAdresseEtablissement']), axis=1)
                     ).compute()

df_2['jacquard'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    jackard_distance(
                     x['adresse_inpi_reconstitue'],
                     x['adress_insee_reconstitue']), axis=1)
                     ).compute()

df_2['edit'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    edit_distance(
                     x['adresse_inpi_reconstitue'],
                     x['adress_insee_reconstitue']), axis=1)
                     ).compute()

df_2['jacquard_enseigne1'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    jackard_distance(
                     x['enseigne'],
                     x['enseigne1Etablissement']), axis=1)
                     ).compute()
df_2['jacquard_enseigne2'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    jackard_distance(
                     x['enseigne'],
                     x['enseigne2Etablissement']), axis=1)
                     ).compute()
df_2['jacquard_enseigne3'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    jackard_distance(
                     x['enseigne'],
                     x['enseigne3Etablissement']), axis=1)
                     ).compute()

df_2['edit_enseigne1'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    edit_distance(
                     x['enseigne'],
                     x['enseigne1Etablissement']), axis=1)
                     ).compute()
df_2['edit_enseigne2'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    edit_distance(
                     x['enseigne'],
                     x['enseigne2Etablissement']), axis=1)
                     ).compute()
df_2['edit_enseigne3'] = df_2.map_partitions(
            lambda df:
                df.apply(lambda x:
                    edit_distance(
                     x['enseigne'],
                     x['enseigne3Etablissement']), axis=1)
                     ).compute()

df_2 = df_2.compute()
## test join Adress
df_2.loc[
        (df_2['test_address_libelle'] == True)
        &(df_2['test_address_complement'] == True),
        'test_join_address'] = True

df_2.loc[
        (df_2['test_join_address'] != True),
        'test_join_address'] = False

df_2 = df_2.assign(
    min_jacquard = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['jacquard'].transform('min'),
    min_edit = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['edit'].transform('min'))

## test join Adress
df_2.loc[
        (df_2['test_address_libelle'] == False)
        &(df_2['test_address_complement'] == False)
        &(df_2['test_join_address'] == False),
        'test_regex_adress'] = False

df_2.loc[
        (df_2['test_regex_adress'] != False),
        'test_regex_adress'] = True

[########################################] | 100% Completed |  2min  5.9s
[########################################] | 100% Completed |  2min  5.0s
[########################################] | 100% Completed | 44.1s
[########################################] | 100% Completed |  4min 20.9s
[########################################] | 100% Completed | 40.6s
[########################################] | 100% Completed | 40.1s
[########################################] | 100% Completed | 40.4s
[########################################] | 100% Completed |  1min 17.8s
[########################################] | 100% Completed | 41.6s
[########################################] | 100% Completed | 41.0s
[########################################] | 100% Completed |  0.5s


In [20]:
df_2 = df_2.assign(
    
    test_distance_diff = lambda x:
    np.where(
        np.logical_or(
            x['jacquard'] != x['min_jacquard'],
            x['edit'] != x['min_edit']    
        ),
        True, False
    
    ),
    
    min_jacquard_enseigne1 = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['jacquard_enseigne1'].transform('min'),
    
    min_jacquard_enseigne2 = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['jacquard_enseigne2'].transform('min'),
    
    min_jacquard_enseigne3 = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['jacquard_enseigne3'].transform('min'),
    
    min_edit_enseigne1 = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['edit_enseigne1'].transform('min'),
    
    min_edit_enseigne2 = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['edit_enseigne2'].transform('min'),
    
    min_edit_enseigne3 = lambda x:
    x.groupby(['siren', 'code_greffe', 'nom_greffe', 'numero_gestion',
               'id_etablissement'])['edit_enseigne3'].transform('min')
)

In [23]:
#df_2 = df_2.drop(columns =  ['test_enseigne_insee','test_enseigne_edit',
#                                               'test_enseigne_jacquard'])

df_2.loc[
    (df_2['enseigne1Etablissement'].isin([np.nan]))
    &(df_2['enseigne2Etablissement'].isin([np.nan]))
    &(df_2['enseigne3Etablissement'].isin([np.nan])),
    'test_enseigne_insee'
] = True

df_2.loc[
        (df_2['test_enseigne_insee'] != True),
        'test_enseigne_insee'] = False

df_2.loc[
    (df_2['enseigne'].isin([np.nan]))
    |(df_2['test_enseigne_insee'].isin([True]))
    |(df_2['jacquard_enseigne1'] == 0)
    |(df_2['jacquard_enseigne2'] == 0)
    |(df_2['jacquard_enseigne3'] == 0),
    'test_enseigne_jacquard'
] = True

df_2.loc[
    (df_2['enseigne'].isin([np.nan]))
    |(df_2['test_enseigne_insee'].isin([True]))
    |(df_2['edit_enseigne1'] == 0)
    |(df_2['edit_enseigne2'] == 0)
    |(df_2['edit_enseigne3'] == 0),
    'test_enseigne_edit'
] = True

df_2.loc[
        (df_2['test_enseigne_edit'] != True),
        'test_enseigne_edit'] = False

df_2.loc[
        (df_2['test_enseigne_jacquard'] != True),
        'test_enseigne_jacquard'] = False

In [24]:
df_2.loc[lambda x: x['siren'].isin(['400534020'])]

,siren,code_greffe,nom_greffe,numero_gestion,id_etablissement,origin,file_timestamp,date_greffe,libelle_evt,last_libele_evt,type,adress_new,adresse_new_clean_reg,voie_matching,numero_voie_matching,code_postal_matching,ncc,code_commune,enseigne,date_début_activité,index,status_admin,status_ets,siret,dateCreationEtablissement,count_initial_insee,etablissementSiege,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,len_digit_address_insee,list_digit_insee,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,INSEE,possibilite,upper,numeroVoieEtablissement_,adresse_insee_clean,adress_insee_reconstitue,adresse_inpi_reconstitue,test_address_libelle,test_address_complement,jacquard,edit,jacquard_enseigne1,jacquard_enseigne2,jacquard_enseigne3,edit_enseigne1,edit_enseigne2,edit_enseigne3,test_join_address,min_jacquard,min_edit,test_regex_adress,test_distance_diff,min_jacquard_enseigne1,min_jacquard_enseigne2,min_jacquard_enseigne3,min_edit_enseigne1,min_edit_enseigne2,min_edit_enseigne3,test_enseigne_insee,test_enseigne_jacquard,test_enseigne_edit
282018,400534020,2501,Besançon,1995B00152,2,Initial,2017-04-28 00:00:00.000,2017-04-28 00:00:00.000,Etablissement ouvert,Etablissement supprimé,SEC,19 RUE DU MARECHAL DE LATTRE DE TASSIGNY,MARECHAL$|LATTRE$|TASSIGNY$,RUE,19,25800,VALDAHON,25578,AUTOCONTROL,2014-02-14,1690148,F,false,40053402000021,2014-02-14,3.0,false,NaN,19,NaN,RUE,DU MARECHAL DE LATTRE DE TASSIGN,0.0,NaN,25800,VALDAHON,NaN,NaN,25578,NaN,NaN,NaN,NaN,F,AUTO TEST,NaN,NaN,RUE,RUE,True,19,MARECHAL LATTRE TASSIGN,19 MARECHAL LATTRE TASSIGN,19 MARECHAL LATTRE TASSIGNY,False,False,0.0625,1,0.636364,NaN,NaN,6.0,NaN,NaN,False,0.0,0,False,True,0.0,NaN,NaN,0.0,NaN,NaN,False,False,False
282019,400534020,2501,Besançon,1995B00152,2,Initial,2017-04-28 00:00:00.000,2017-04-28 00:00:00.000,Etablissement ouvert,Etablissement supprimé,SEC,19 RUE DU MARECHAL DE LATTRE DE TASSIGNY,MARECHAL$|LATTRE$|TASSIGNY$,RUE,19,25800,VALDAHON,25578,AUTOCONTROL,2014-02-14,1690148,F,false,40053402000039,2014-02-14,3.0,false,NaN,19,NaN,RUE,MARECHAL DE LATTRE DE TASSIGNY,0.0,NaN,25800,VALDAHON,NaN,NaN,25578,NaN,NaN,NaN,NaN,F,AUTO TEST,NaN,NaN,RUE,RUE,True,19,MARECHAL LATTRE TASSIGNY,19 MARECHAL LATTRE TASSIGNY,19 MARECHAL LATTRE TASSIGNY,True,False,0.0000,0,0.636364,NaN,NaN,6.0,NaN,NaN,False,0.0,0,True,False,0.0,NaN,NaN,0.0,NaN,NaN,False,False,False
282020,400534020,2501,Besançon,1995B00152,2,EVT,2017-05-13 06:41:09.000,2017-05-12 00:00:00.000,Modifications relatives à un établissement,Etablissement supprimé,SEC,19 RUE DU MARECHAL DE LATTRE DE TASSIGN,MARECHAL$|LATTRE$|TASSIGN$,RUE,19,25800,VALDAHON,25578,AUTO TEST,2014-02-14,1690149,F,false,40053402000021,2014-02-14,3.0,false,NaN,19,NaN,RUE,DU MARECHAL DE LATTRE DE TASSIGN,0.0,NaN,25800,VALDAHON,NaN,NaN,25578,NaN,NaN,NaN,NaN,F,AUTO TEST,NaN,NaN,RUE,RUE,True,19,MARECHAL LATTRE TASSIGN,19 MARECHAL LATTRE TASSIGN,19 MARECHAL LATTRE TASSIGN,True,False,0.0000,0,0.000000,NaN,NaN,0.0,NaN,NaN,False,0.0,0,True,False,0.0,NaN,NaN,0.0,NaN,NaN,False,True,True
282021,400534020,2501,Besançon,1995B00152,2,EVT,2017-05-13 06:41:09.000,2017-05-12 00:00:00.000,Modifications relatives à un établissement,Etablissement supprimé,SEC,19 RUE DU MARECHAL DE LATTRE DE TASSIGN,MARECHAL$|LATTRE$|TASSIGN$,RUE,19,25800,VALDAHON,25578,AUTO TEST,2014-02-14,1690149,F,false,40053402000039,2014-02-14,3.0,false,NaN,19,NaN,RUE,MARECHAL DE LATTRE DE TASSIGNY,0.0,NaN,25800,VALDAHON,NaN,NaN,25578,NaN,NaN,NaN,NaN,F,AUTO TEST,NaN,NaN,RUE,RUE,True,19,MARECHAL LATTRE TASSIGNY,19 MARECHAL LATTRE TASSIGNY,19 MARECHAL LATTRE TASSIGN,False,False,0.0625,1,0.000000,NaN,NaN,0.0,NaN,N

In [25]:
df_2.head()

,siren,code_greffe,nom_greffe,numero_gestion,id_etablissement,origin,file_timestamp,date_greffe,libelle_evt,last_libele_evt,type,adress_new,adresse_new_clean_reg,voie_matching,numero_voie_matching,code_postal_matching,ncc,code_commune,enseigne,date_début_activité,index,status_admin,status_ets,siret,dateCreationEtablissement,count_initial_insee,etablissementSiege,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,len_digit_address_insee,list_digit_insee,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,INSEE,possibilite,upper,numeroVoieEtablissement_,adresse_insee_clean,adress_insee_reconstitue,adresse_inpi_reconstitue,test_address_libelle,test_address_complement,jacquard,edit,jacquard_enseigne1,jacquard_enseigne2,jacquard_enseigne3,edit_enseigne1,edit_enseigne2,edit_enseigne3,test_join_address,min_jacquard,min_edit,test_regex_adress,test_distance_diff,min_jacquard_enseigne1,min_jacquard_enseigne2,min_jacquard_enseigne3,min_edit_enseigne1,min_edit_enseigne2,min_edit_enseigne3,test_enseigne_insee,test_enseigne_jacquard,test_enseigne_edit
0,511830093,1104,Narbonne,2009D00047,1,Initial,2014-05-09 00:00:00.000,2014-05-09 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SEP,3 ZA LA VIGNASSE,ZA$|VIGNASSE$,NaN,3,11440,PEYRIAC DE MER,11285,NaN,2009-04-07,4757027,A,true,51183009300015,2009-04-07,1.0,true,NaN,3,NaN,NaN,ZA LA VIGNASSE,0.0,NaN,11440,PEYRIAC DE MER,NaN,NaN,11285,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,,NaN,3,ZA VIGNASSE,3 ZA VIGNASSE,3 ZA VIGNASSE,True,False,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
1,511830663,4801,Mende,2009B00049,1,Initial,2011-10-26 00:00:00.000,2011-10-26 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SEP,RUE DU PETIT TOUR DE VILLE,PETIT$|TOUR$|VILLE$,RUE,NaN,48300,LANGOGNE,48080,NaN,2009-04-01,4757043,A,true,51183066300015,2009-04-01,2.0,true,NaN,NaN,NaN,RUE,DU PETIT TOUR DE VILLE,0.0,NaN,48300,LANGOGNE,NaN,NaN,48080,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,RUE,RUE,True,,PETIT TOUR VILLE,PETIT TOUR VILLE,PETIT TOUR VILLE,True,False,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
2,511830663,4801,Mende,2009B00049,2,Initial,2011-10-26 00:00:00.000,2011-10-26 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SEC,22 BOULEVARD DE GAULLE,GAULLE$,BD,22,48300,LANGOGNE,48080,NaN,2011-09-14,4757044,A,false,51183066300023,2011-09-14,2.0,false,NaN,22,NaN,BD,DE GAULLE,0.0,NaN,48300,LANGOGNE,NaN,NaN,48080,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,BD,BOULEVARD,True,22,GAULLE,22 GAULLE,22 GAULLE,True,False,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
3,511830788,8801,Epinal,2009B00237,1,Initial,2016-10-17 00:00:00.000,2016-10-17 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SEP,32 ROUTE DU QUARTIER,NaN,RTE,32,88310,CORNIMONT,88116,NaN,2009-04-01,4757046,A,true,51183078800010,2009-04-01,1.0,true,NaN,32,NaN,RTE,DU QUARTIER,0.0,NaN,88310,CORNIMONT,NaN,NaN,88116,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,RTE,ROUTE,True,32,,32,32,False,False,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
4,511830952,2202,Saint-Brieuc,2009D00229,1,Initial,2010-03-23 00:00:00.000,2010-03-23 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SEP,LE BOURG,BOURG$,NaN,NaN,22200,MOUSTERU,22156,NaN,2009-03-06,4757058,A,true,51183095200012,2009-03-06,1.0,true,NaN,NaN,NaN,NaN,BOURG,0.0,NaN,22200,MOUSTERU,NaN,NaN,22156,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,,NaN,,BOURG,BOURG,BOURG,True,False,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True


In [26]:
### Nombre de duplicate
df_2.shape[0] -df_2['index'].nunique()

9095

## Etape 5: Selection meilleurs candidats

Dans cette dernière étape, on ne garde que les valeurs ont les tests ont été concluant sur le regex de l'adresse

In [ ]:
#count_conformite_test(df = df_2)

In [27]:
test = split_duplication(df = df_2)

In [28]:
test['not_duplication'].shape

(1352514, 79)

In [29]:
test['duplication'].shape

(13691, 79)

In [ ]:
test['duplication'].head(10)

## Recupération

In [30]:
test['duplication']['index'].nunique()

4596

In [31]:
df_filre = test['duplication'].copy()
index_found = pd.DataFrame()
df_filre.shape#.head()

(13691, 79)

In [32]:
for i in ['test_join_address', 'test_address_libelle',
          'test_address_complement', 'jacquard', 'edit']:
        
        if i in ['jacquard']:
            
            index_to_keep = split_duplication(
                (df_filre.loc[lambda x:                     
                                             (x[i] == x['min_jacquard'])
                                              ]
                                             )
                                             )['not_duplication']#['index']
            
        elif i in ['edit']:
            
            index_to_keep = split_duplication(
                (df_filre.loc[lambda x:             
                                             (x[i] == x['min_edit'])
                                              ]
                                             )
                                             )['not_duplication']#['index']
     
        else:
            
            index_to_keep = split_duplication(
                (df_filre.loc[lambda x: 
                                             (x[i].isin([True]))
                                            ]
                                             )
                                             )['not_duplication']#['index']
                
        index_found = index_found.append(index_to_keep)
        df_filre = df_filre.loc[lambda x: (~x['index'].isin(index_to_keep['index'].values))]

In [33]:
df_filre.shape

(6058, 79)

In [34]:
index_found.shape

(2898, 79)

In [35]:
index_found['index'].nunique() + df_filre['index'].nunique() == test['duplication']['index'].nunique()

True

## Règles de conformité

- Test 1: - > ?
    - Plusieurs SIRET/sequence
- Test 2:
    - Ensemble des regex faux
        - `test_regex_adress`
- Test 3
    - Test sur l'enseigne
        - Si `test_enseigne_edit` = True (-> Enseigne INSEE & INPI, match parfait)
        - Si `test_enseigne_jacquard` = True
- Test 4:
    - Test sur la distance:
        - Si `test_distance_diff` =  True & `min_jacquard` < .2 
        - Si `test_distance_diff` =  True & `min_edit` < 20 

In [ ]:
import sidetable

In [38]:
df_conformite = (pd.concat(
[test['not_duplication'],
 index_found
]#, axis = 1
))

In [40]:
df_conformite.shape

(1355412, 79)

In [ ]:
sequence = ['siren', 'code_greffe', 'nom_greffe', 'numero_gestion', 'id_etablissement']
df_conformite = df_conformite.assign(
    total_siret = lambda x: x.groupby(sequence)['siret'].transform('nunique')
)
df_conformite.loc[lambda x: x['total_siret'].isin([2])].to_excel('plusieurs_siret.xlsx')

In [ ]:
#df_conformite.dtypes.to_frame().T#.unstack()

In [49]:
#df_conformite.loc[lambda x: x['siren'].isin(['451041172'])]

In [46]:
#df_conformite.loc[lambda x: x['test_enseigne_jacquard'].isin([False])]

In [51]:
df_conformite.stb.freq(['test_regex_adress'])

,test_regex_adress,count,percent,cumulative_count,cumulative_percent
0,True,1308777,96.559349,1308777,96.559349
1,False,46635,3.440651,1355412,100.000000


In [52]:
df_conformite.stb.freq(['test_distance_diff'])

,test_distance_diff,count,percent,cumulative_count,cumulative_percent
0,False,1350003,99.600933,1350003,99.600933
1,True,5409,0.399067,1355412,100.000000


In [42]:
df_conformite.stb.freq(['test_enseigne_jacquard'])

,test_enseigne_jacquard,count,percent,cumulative_count,cumulative_percent
0,True,1336660,98.616509,1336660,98.616509
1,False,18752,1.383491,1355412,100.000000


In [43]:
df_conformite.stb.freq(['test_enseigne_edit'])

,test_enseigne_edit,count,percent,cumulative_count,cumulative_percent
0,True,1333105,98.354227,1333105,98.354227
1,False,22307,1.645773,1355412,100.000000


### Filtrage

In [64]:
df_conformite.shape

(1355412, 80)

In [175]:
test_1 = (df_conformite.loc[
    
    lambda x: 
    (x['test_regex_adress'].isin([True]))
    &
    (x['test_enseigne_edit'].isin([True]))
    |
    (x['test_enseigne_jacquard'].isin([True]))
    |
    (x['count_initial_insee'].isin([1]))
    & 
    (x['total_siret'].isin([1]))
    #&
    #(x['test_distance_diff'].isin([False])) 
    #& ((x['min_jacquard'] < .2) 
    # | (x['min_edit'] < 20))
]#['index']
)

Besoin de compter le nombre de ligne à l'INSEE pour les variables utilisées pour matcher

In [128]:
#df_conformite.loc[lambda x: (x['siren'].isin(['998823504'])) 
#           & (x['id_etablissement'].isin(['185']))
#                      ].sort_values(by = ['id_etablissement'])

In [176]:
df_conformite.shape[0] - len(test_1)

13638

In [178]:
test_1[sequence].drop_duplicates() 

,siren,siret,code_greffe,nom_greffe,numero_gestion,id_etablissement,total_siret
0,511830093,51183009300015,1104,Narbonne,2009D00047,1,1
1,511830663,51183066300015,4801,Mende,2009B00049,1,1
2,511830663,51183066300023,4801,Mende,2009B00049,2,1
3,511830788,51183078800010,8801,Epinal,2009B00237,1,1
4,511830952,51183095200012,2202,Saint-Brieuc,2009D00229,1,1
...,...,...,...,...,...,...,...
5250,814075735,81407573500062,5201,Chaumont,2019B00123,3,1
5441,848125449,84812544900043,2104,Dijon,2019B00379,3,1
5600,833546120,83354612000027,3601,Châteauroux,2018B00254,1,1
5602,833546120,83354612000035,3601,Châteauroux,2018B00254,2,1


Recupération de l'historique

In [184]:
df_to_match = inpi.compute().loc[lambda x: 
                                 ~x['index'].isin(df_conformite['index'])
                                ]
df_to_match.shape

[########################################] | 100% Completed | 49.5s


(9293133, 23)

In [185]:
df_to_match.head()

,siren,code_greffe,nom_greffe,numero_gestion,id_etablissement,origin,file_timestamp,date_greffe,libelle_evt,last_libele_evt,type,adress_new,adresse_new_clean_reg,voie_matching,numero_voie_matching,code_postal_matching,ncc,code_commune,enseigne,date_début_activité,index,status_admin,status_ets
0,511830044,2602,Romans,2009B00486,4,Initial,2015-06-17 00:00:00.000,2015-06-17 00:00:00.000,Etablissement ouvert,Etablissement ouvert,PRI,10 GRANDE RUE,GRANDE$,GR,10,26130,SAINT PAUL TROIS CHATEAUX,26324,NaN,2009-05-01,4757025,A,false
1,511830077,5103,Reims,2009A00092,1,Initial,2017-05-04 00:00:00.000,2017-05-04 00:00:00.000,Etablissement ouvert,Etablissement ouvert,PRI,4 RUE DES PRES,PRES$,RUE,4,51270,CONGY,51163,NaN,2009-05-15,4757026,A,false
3,511830135,9201,Nanterre,2009B02557,1,Initial,2016-07-26 00:00:00.000,2016-07-26 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SIE,23 AVENUE JOFFRE,JOFFRE$,AV,23,92250,GARENNE COLOMBES,92035,NaN,NaN,4757028,A,true
4,511830135,9201,Nanterre,2009B02557,2,Initial,2016-07-26 00:00:00.000,2016-07-26 00:00:00.000,Etablissement ouvert,Etablissement ouvert,PRI,23 AVENUE JOFFRE,JOFFRE$,AV,23,92250,GARENNE COLOMBES,92035,NaN,2009-04-01,4757029,A,false
5,511830143,2501,Besançon,2009D00107,0,Initial,2016-03-02 00:00:00.000,2016-03-02 00:00:00.000,Etablissement ouvert,Etablissement ouvert,SIE,17 RUE ALAIN SAVARY TEMIS CENTER 1,ALAIN$|SAVARY$|TEMIS$|CENTER$,RUE,17,25000,BESANCON,25056,NaN,NaN,4757030,A,true


In [186]:
seq = ['siren','code_greffe', 'nom_greffe', 'numero_gestion', 'id_etablissement']
sequence = ['siren', 'siret','code_greffe', 'nom_greffe', 'numero_gestion', 'id_etablissement', 'total_siret']

columns_to_keep = ['siren', 'siret', 'code_greffe', 'nom_greffe', 'numero_gestion',
       'id_etablissement', 'total_siret', 'origin', 'file_timestamp',
       'date_greffe', 'libelle_evt', 'last_libele_evt', 'type', 'adress_new',
       'adresse_new_clean_reg', 'voie_matching', 'numero_voie_matching',
       'code_postal_matching', 'ncc', 'code_commune', 'enseigne',
       'date_début_activité', 'index', 'status_admin', 'status_ets', '_merge']

df_match_2 = (
    (test_1[sequence]
 .drop_duplicates()  
    )
 .merge(df_to_match, right_on = seq, left_on = seq, how = 'left', indicator = True)
)

In [187]:
df_match_2.stb.freq(['_merge'])

,_merge,count,percent,cumulative_count,cumulative_percent
0,left_only,1197317,95.192123,1197317,95.192123
1,both,60473,4.807877,1257790,100.000000


In [188]:
df_match_2.head()

,siren,siret,code_greffe,nom_greffe,numero_gestion,id_etablissement,total_siret,origin,file_timestamp,date_greffe,libelle_evt,last_libele_evt,type,adress_new,adresse_new_clean_reg,voie_matching,numero_voie_matching,code_postal_matching,ncc,code_commune,enseigne,date_début_activité,index,status_admin,status_ets,_merge
0,511830093,51183009300015,1104,Narbonne,2009D00047,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,511830663,51183066300015,4801,Mende,2009B00049,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,511830663,51183066300023,4801,Mende,2009B00049,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,511830788,51183078800010,8801,Epinal,2009B00237,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,511830952,51183095200012,2202,Saint-Brieuc,2009D00229,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [134]:
#df_match_2.loc[lambda x: x['_merge'].isin(['both'])].dtypes

In [189]:
df_final = (pd.concat(
    [
        df_match_2.loc[lambda x: x['_merge'].isin(['both'])],
        df_conformite.reindex(columns  = columns_to_keep)
        
    ])
 .sort_values(by = [
     'siren','code_greffe', 'nom_greffe', 'numero_gestion', 'id_etablissement', 'date_greffe'
 ])
)

In [148]:
df_final.groupby("index")['index'].count().sort_values().loc[lambda x: x>1]

index
7357852.0     2
9927603.0     2
8145131.0     2
8753389.0     2
8548195.0     2
6853200.0     2
10062463.0    2
436876.0      2
Name: index, dtype: int64

In [193]:
df_conformite.loc[lambda x: x['index'].isin(['000735785'])]

,siren,code_greffe,nom_greffe,numero_gestion,id_etablissement,origin,file_timestamp,date_greffe,libelle_evt,last_libele_evt,type,adress_new,adresse_new_clean_reg,voie_matching,numero_voie_matching,code_postal_matching,ncc,code_commune,enseigne,date_début_activité,index,status_admin,status_ets,siret,dateCreationEtablissement,count_initial_insee,etablissementSiege,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,len_digit_address_insee,list_digit_insee,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,INSEE,possibilite,upper,numeroVoieEtablissement_,adresse_insee_clean,adress_insee_reconstitue,adresse_inpi_reconstitue,test_address_libelle,test_address_complement,jacquard,edit,jacquard_enseigne1,jacquard_enseigne2,jacquard_enseigne3,edit_enseigne1,edit_enseigne2,edit_enseigne3,test_join_address,min_jacquard,min_edit,test_regex_adress,test_distance_diff,min_jacquard_enseigne1,min_jacquard_enseigne2,min_jacquard_enseigne3,min_edit_enseigne1,min_edit_enseigne2,min_edit_enseigne3,test_enseigne_insee,test_enseigne_jacquard,test_enseigne_edit,count_duplicates_,total_siret


In [153]:
#df_conformite.loc[lambda x: x['siren'].isin(['814217535'])]

In [198]:
test_1.loc[lambda x: x['siren'].isin(['814217535'])]

,siren,code_greffe,nom_greffe,numero_gestion,id_etablissement,origin,file_timestamp,date_greffe,libelle_evt,last_libele_evt,type,adress_new,adresse_new_clean_reg,voie_matching,numero_voie_matching,code_postal_matching,ncc,code_commune,enseigne,date_début_activité,index,status_admin,status_ets,siret,dateCreationEtablissement,count_initial_insee,etablissementSiege,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,len_digit_address_insee,list_digit_insee,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,INSEE,possibilite,upper,numeroVoieEtablissement_,adresse_insee_clean,adress_insee_reconstitue,adresse_inpi_reconstitue,test_address_libelle,test_address_complement,jacquard,edit,jacquard_enseigne1,jacquard_enseigne2,jacquard_enseigne3,edit_enseigne1,edit_enseigne2,edit_enseigne3,test_join_address,min_jacquard,min_edit,test_regex_adress,test_distance_diff,min_jacquard_enseigne1,min_jacquard_enseigne2,min_jacquard_enseigne3,min_edit_enseigne1,min_edit_enseigne2,min_edit_enseigne3,test_enseigne_insee,test_enseigne_jacquard,test_enseigne_edit,count_duplicates_,total_siret
853635,814217535,6401,Bayonne,2015B00921,1,Initial,2016-09-05 00:00:00.000,2016-09-05 00:00:00.000,Etablissement ouvert,Modifications relatives à un établissement,SEP,24 AVENUE ARGI EDER,ARGI$|EDER$,AV,24,64500,SAINT JEAN DE LUZ,64483,NaN,2015-10-01,7357851,A,true,81421753500016,2015-10-01,3.0,true,NaN,24,NaN,AV,ARGI EDER,0.0,NaN,64500,SAINT JEAN DE LUZ,NaN,NaN,64483,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,AV,AVENUE,True,24,ARGI EDER,24 ARGI EDER,24 ARGI EDER,True,False,0.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,False,0.0,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,1,2
853636,814217535,6401,Bayonne,2015B00921,1,EVT,2019-09-19 09:33:49.000,2019-09-18 00:00:00.000,Modifications relatives à un établissement,Modifications relatives à un établissement,PRI,ALL MONTAURY BATIMENT LE FUTURA 62 AVENUE DE B...,ALL$|MONTAURY$|BATIMENT$|FUTURA$|BAYONNE$,AV,62,64600,ANGLET,64024,NaN,2019-02-22,7357853,A,false,81421753500032,2019-02-22,3.0,false,ALL MONTAURY BAT LE FUTURA,62,NaN,AV,DE BAYONNE,0.0,NaN,64600,ANGLET,NaN,NaN,64024,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,AV,AVENUE,True,62,BAYONNE,62 BAYONNE,ALL MONTAURY BATIMENT FUTURA 62 BAYONNE,True,True,0.4375,29,NaN,NaN,NaN,NaN,NaN,NaN,True,0.0,0,True,True,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,1,2


In [201]:
split_duplication(
                df_final)['not_duplication'].shape

(1415869, 27)

In [202]:
split_duplication(
                df_final)['duplication'].shape

(16, 27)